# Enunciado del Problema:
Como parte de su estrategia de transformación digital, el Banco Digital de Chile (BDC) ha formado un selecto grupo de profesionales, del cual usted es parte, para implementar una serie de proyectos de inteligencia artificial y analítica. 
Su rol, dentro de la organización, es apoyar a los científicos de datos ya que ellos no tienen desarrolladas sus habilidades de Data Wrangling y necesitan un estrecho apoyo de los ingenieros de datos para llevar a cabo sus modelos de inteligencia artificial.
El proyecto actual en el que se encuentra participando, requiere que se construya una base de datos de información de la competencia en cuanto a las captaciones, colocaciones e inversiones a lo largo del tiempo. Esta tarea era desarrollada de forma manual por los analistas, quienes se conectaban a la página del Banco Central y descargaban las planillas Excel para alimentar los datos, pero se cometían muchos errores de transcripción, razón por la cual le han encomendado a usted que automatice un proceso de obtención y preparación de datos.
A continuación, se presenta el link para la descarga de los archivos que utilizan los analistas:
https://www.bcentral.cl/contenido/-/detalle/serie-de-datos-bancarios-septiembre-de-2021
Nótese que este link podría variar durante el módulo, por lo tanto, deberá buscar la información disponible siguiendo la navegación sugerida en el breadcrum siguiente.

# Requisitos del programa:
Construir un programa en Python para la extracción y preparación de datos ya sea de colocaciones, depósitos e inversiones disponible en la página web del Banco Central. 
Recuerde que debe utilizar las técnicas y buenas prácticas aprendidas en clase en lo que respecta a limpieza de datos y data wrangling.
Los entregables de su trabajo son los siguientes:
- Un archivo csv que tenga la información consolidada del resultado
- El notebook de Python donde realiza la obtención y preparación de la data

El archivo de salida debe considerar las siguientes columnas:
- Fecha
- Institución Financiera
- Tipo (colocación, captación, inversión)
- Subtipo
- Valor
- Tipo de moneda

En el caso de los depósitos, se podrá dar cuenta que hay distintos tipos de colocaciones en las distintas hojas de la planilla, por ejemplo, colocaciones comerciales, colocaciones de consumo, de vivienda, de comercio exterior, etc. Utilice esta información para fijar el campo Subtipo solicitado.
También se podrá dar cuenta que algunas colocaciones están expresadas en dólares, con lo cual podrá alimentar el campo Tipo de Moneda. Lo mismo aplica para los otros dos archivos, es decir, de depósito y de inversiones.
Haga todos los supuestos que estime conveniente y déjelos bien documentados en su notebook. Asuma que el formato de los archivos es siempre el mismo y no cambia con el tiempo, por lo tanto usted podría referenciar una hoja particular sabiendo que se trata de un tipo o subtipo específico.
Haga todas las simplificaciones que estime conveniente para lograr el objetivo, lo importante es que los datos que contenga el archivo de salida sean consistentes.

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Procesar los archivos excel

def procesar_archivo_excel(ruta_archivo, tipo_dato):
    # Leer todas las hojas del archivo
    excel_file = pd.ExcelFile(ruta_archivo)
    
    # Lista para almacenar los dataframes de cada hoja
    datos_consolidados = []
    
    # Procesar cada hoja
    for nombre_hoja in excel_file.sheet_names:
        print(f"Procesando hoja: {nombre_hoja} del archivo {os.path.basename(ruta_archivo)}")
        
        # Leer la hoja completa
        df_hoja = pd.read_excel(ruta_archivo, sheet_name=nombre_hoja, header=None)
        
        # Extraer metadatos
        subtipo = df_hoja.iloc[2, 1]  # Celda B3
        moneda_texto = df_hoja.iloc[3, 1]  # Celda B4
        
        # Determinar tipo de moneda
        if 'dólares' in str(moneda_texto).lower() or 'dolares' in str(moneda_texto).lower():
            tipo_moneda = 'USD'
        else:
            tipo_moneda = 'CLP'
        
        # Extraer fechas // fila 6 (índice 5), desde columna C (índice 2)
        fechas = df_hoja.iloc[5, 2:].values
        
        # Extraer instituciones financieras // filas 7-32 (índices 6-31), columna B (índice 1)
        instituciones = df_hoja.iloc[6:32, 1].values  # F
        
        # Extraer valores // Filas 7-32, desde columna C
        valores = df_hoja.iloc[6:32, 2:].values
        
        # Crear lista para almacenar registros
        registros = []
        
        # Iterar sobre las instituciones y valores
        for i, institucion in enumerate(instituciones):
            for j, fecha in enumerate(fechas):
                if j < valores.shape[1]:
                    valor = valores[i, j]
                    if pd.notna(valor) and pd.notna(institucion) and pd.notna(fecha):
                        registros.append({
                            'Fecha': fecha,
                            'Institución Financiera': institucion,
                            'Tipo': tipo_dato,
                            'Subtipo': subtipo,
                            'Valor (millones)': valor,
                            'Tipo de moneda': tipo_moneda
                        })
        
        # Crear df de la hoja actual
        if registros:
            df_registros = pd.DataFrame(registros)
            datos_consolidados.append(df_registros)
    
    # Consolidar todos los df
    if datos_consolidados:
        df_final = pd.concat(datos_consolidados, ignore_index=True)
        return df_final
    else:
        return pd.DataFrame()

In [4]:
def consolidar_archivos_bancarios(ruta_colocacion, ruta_captacion, ruta_inversion, ruta_salida='datos_consolidados.xlsx'):
    print("Iniciando proceso de consolidación...")
    print("-" * 60)
    
    # Procesar cada archivo
    df_colocacion = procesar_archivo_excel(ruta_colocacion, 'Colocación')
    print(f"\nColocación procesada: {len(df_colocacion)} registros")
    
    df_captacion = procesar_archivo_excel(ruta_captacion, 'Captación')
    print(f"Captación procesada: {len(df_captacion)} registros")
    
    df_inversion = procesar_archivo_excel(ruta_inversion, 'Inversión')
    print(f"Inversión procesada: {len(df_inversion)} registros")
    
    # Consolidar todos los datos
    df_consolidado = pd.concat([df_colocacion, df_captacion, df_inversion], 
                                ignore_index=True)
    
    # Convertir fechas al formato adecuado
    df_consolidado['Fecha'] = pd.to_datetime(df_consolidado['Fecha'], errors='coerce')
    
    # Ordenar por fecha y tipo
    df_consolidado = df_consolidado.sort_values(['Fecha', 'Tipo', 'Institución Financiera'])
    
    # Guardar en Excel
    df_consolidado.to_excel(ruta_salida, index=False, engine='openpyxl')
    
    print("-" * 60)
    print(f"\nProceso completado!")
    print(f"Total de registros consolidados: {len(df_consolidado)}")
    print(f"Archivo guardado en: {ruta_salida}")
    
    # Mostrar resumen
    print("\nResumen por tipo:")
    print(df_consolidado.groupby('Tipo').size())
    
    print("\nResumen por moneda:")
    print(df_consolidado.groupby('Tipo de moneda').size())
    
    return df_consolidado

In [5]:
if __name__ == "__main__":
    # Definir rutas de archivos
    archivo_colocacion = 'sdbcol_092021.xlsx'
    archivo_captacion = 'sdbdep_092021.xlsx'
    archivo_inversion = 'sdbinv_092021.xlsx'
    
    # Ejecutar consolidación
    df_resultado = consolidar_archivos_bancarios(
        ruta_colocacion=archivo_colocacion,
        ruta_captacion=archivo_captacion,
        ruta_inversion=archivo_inversion,
        ruta_salida='datos_bancarios_consolidados.xlsx'
    )

Iniciando proceso de consolidación...
------------------------------------------------------------
Procesando hoja: Inicio_Colocaciones del archivo sdbcol_092021.xlsx
Procesando hoja: Índice_general del archivo sdbcol_092021.xlsx
Procesando hoja: Notas_generales del archivo sdbcol_092021.xlsx
Procesando hoja: Glosario del archivo sdbcol_092021.xlsx
Procesando hoja: 1_01 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_02 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_03 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_04 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_05 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_06 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_07 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_08 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_09 del archivo sdbcol_092021.xlsx
Procesando hoja: 1_10 del archivo sdbcol_092021.xlsx

Colocación procesada: 42944 registros
Procesando hoja: Inicio_Depósitos del archivo sdbdep_092021.xlsx
Procesando hoja

/tmp/ipykernel_4561/2237292335.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_consolidado['Fecha'] = pd.to_datetime(df_consolidado['Fecha'], errors='coerce')


------------------------------------------------------------

Proceso completado!
Total de registros consolidados: 137414
Archivo guardado en: datos_bancarios_consolidados.xlsx

Resumen por tipo:
Tipo
Captación     38655
Colocación    42944
Inversión     55815
dtype: int64

Resumen por moneda:
Tipo de moneda
CLP    115964
USD     21450
dtype: int64


In [6]:
df_resultado

,Fecha,Institución Financiera,Tipo,Subtipo,Valor (millones),Tipo de moneda
43154,2008-01-31,Banco BTG Pactual Chile (9),Captación,Total de depósitos y captaciones en moneda nac...,ND,CLP
47444,2008-01-31,Banco BTG Pactual Chile (9),Captación,Total de depósitos en moneda nacional,ND,CLP
51734,2008-01-31,Banco BTG Pactual Chile (9),Captación,Depósitos a menos de un mes,ND,CLP
56024,2008-01-31,Banco BTG Pactual Chile (9),Captación,Depósitos a más de un mes,ND,CLP
60314,2008-01-31,Banco BTG Pactual Chile (9),Captación,Ventas de títulos con pacto de retrocompra,ND,CLP
...,...,...,...,...,...,...
81639,NaT,Otros documentos emitidos por organismos fisc...,Inversión,Concepto,115018; 135018; 136018,CLP
81640,NaT,Otros documentos emitidos por otras institucio...,Inversión,Concepto,Corresponden a instrumentos financieros emitid...,CLP
81641,NaT,Otros documentos emitidos por otras institucio...,Inversión,Concepto,1150201; 1350201; 1360201,CLP
81630,NaT,Venta de títulos con pacto de retrocompra,Inversión,Concepto,Corresponden a operaciones en que la instituci...,CLP


In [ ]:
# Verificar tipos de datos
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137414 entries, 43154 to 81631
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Fecha                   137280 non-null  datetime64[ns]
 1   Institución Financiera  137414 non-null  object        
 2   Tipo                    137414 non-null  object        
 3   Subtipo                 137378 non-null  object        
 4   Valor (millones)        137414 non-null  object        
 5   Tipo de moneda          137414 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 7.3+ MB


In [8]:
# Cambia valores ND por NaN
df_resultado['Valor (millones)'] = df_resultado['Valor (millones)'].replace("ND", np.nan)

In [9]:
# Elimina los registros si fecha
df_resultado = df_resultado.dropna(subset=['Fecha'])

In [10]:
# Cambia el tipo de dato a float
df_resultado['Valor (millones)'] = df_resultado['Valor (millones)'].astype(float)

/tmp/ipykernel_4561/633329691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado['Valor (millones)'] = df_resultado['Valor (millones)'].astype(float)


In [11]:
# Verificar los cambios
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137280 entries, 43154 to 137248
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Fecha                   137280 non-null  datetime64[ns]
 1   Institución Financiera  137280 non-null  object        
 2   Tipo                    137280 non-null  object        
 3   Subtipo                 137280 non-null  object        
 4   Valor (millones)        110342 non-null  float64       
 5   Tipo de moneda          137280 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 7.3+ MB


In [12]:
df_resultado.head()

,Fecha,Institución Financiera,Tipo,Subtipo,Valor (millones),Tipo de moneda
43154,2008-01-31,Banco BTG Pactual Chile (9),Captación,Total de depósitos y captaciones en moneda nac...,NaN,CLP
47444,2008-01-31,Banco BTG Pactual Chile (9),Captación,Total de depósitos en moneda nacional,NaN,CLP
51734,2008-01-31,Banco BTG Pactual Chile (9),Captación,Depósitos a menos de un mes,NaN,CLP
56024,2008-01-31,Banco BTG Pactual Chile (9),Captación,Depósitos a más de un mes,NaN,CLP
60314,2008-01-31,Banco BTG Pactual Chile (9),Captación,Ventas de títulos con pacto de retrocompra,NaN,CLP


In [13]:
# Exportar el DataFrame df_resultado
df_resultado.to_csv('datos_bancarios_consolidados.csv', sep=';', index=False, encoding='utf-8-sig')   
# NOTA: encoding='utf-8-sig asegura que Excel abra tildes y eñes correctamente

print("¡Archivo exportado exitosamente como 'datos_bancarios_consolidados.csv'!")

¡Archivo exportado exitosamente como 'datos_bancarios_consolidados.csv'!
